# Making A Structured TTRPG Story with simpleaichat

An update to ChatGPT on June 13th, 2023 allows the user to set a predefined schema to have ChatGPT output data according to that schema and/or take in an input schema and respond better to that data. This "function calling" as OpenAI calls it can be used as a form of tools, but the schema, enabled by a JSON-finetuning of ChatGPT, is much more useful for typical generative AI use cases, particularly when not using GPT-4.

OpenAI's [official demos](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_call_functions_with_chat_models.ipynb) for this feature are complicated, but with simpleaichat, it's very easy to support placing your own data

**NOTE: Ensuring input and output follows a complex predefined structure is very new in the field of prompt engineering and although it is very powerful, your mileage may vary.**


In [23]:
!pip install -q simpleaichat

from simpleaichat import AIChat
import orjson
from rich.console import Console

from typing import List, Literal, Optional, Union
from pydantic import BaseModel, Field

For the following cell, input your OpenAI API key when prompted. **It will not be saved to the notebook**.

In [ ]:
api_key = getpass("OpenAI Key: ")

## Creating a TTRPG the old-fashioned ChatGPT way

Let's first create a TTRPG setting using the typical workflows of simpleaichat and ChatGPT with system prompt engineering.

For this demo, we'll create a TTRPG about **Python software development** and **beach volleyball**. 

Yes, really. At the least, the resulting TTRPG will be _unique_.


In [2]:
system_prompt = """You are a world-renowned game master (GM) of tabletop role-playing games (RPGs).

Write a setting description and two character sheets for the setting the user provides.

Rules you MUST follow:
- Always write in the style of 80's fantasy novels.
- All names you create must be creative and unique. Always subvert expectations.
- Include as much information as possible in your response."""

In [3]:
model = "gpt-3.5-turbo-0613"
ai = AIChat(system=system_prompt, model=model, save_messages=False, api_key=api_key)

In [4]:
response = ai("Python software development and beach volleyball")
print(response)

The Legend of Zephyrus: Sands of Serpentia

Welcome, brave adventurers, to the mystical realm of Serpentia, a land where the art of Python software development intertwines with the fierce battles of beach volleyball. In this enchanting world, the ancient deity Zephyrus, the God of Wind, has bestowed upon the chosen few the ability to harness the power of code and the skill of volleyball to protect the realm from the encroaching forces of darkness.

Setting Description:

Serpentia is a vibrant land filled with lush palm trees, golden sandy beaches, and crystal-clear turquoise waters. The sun shines brightly overhead, casting a warm glow upon the land. The realm is divided into three main regions:

1. Codehaven: Nestled amidst the towering palm trees, Codehaven is a bustling city where the art of Python software development thrives. Here, the streets are lined with grand libraries and bustling marketplaces, where scribes and scholars exchange knowledge and trade powerful artifacts imbued

Evocative, but a bit disorganized. If we instead allow for structured data output that follows specifications, then we'll have a lot more flexibility both in terms of directing generation, and playing with the resulting output.

That's where the `schema_output` field comes in when generating. If you construct a schema with pydantic ,which is also installed with simpleaichat as it is used heavily internally, then the output will generally follow the schema you provide!

We want an output containing the setting **name** and **description**, along with a list of player characters. Since each character has its own attributes, and we may want the model to generate multiple chatacters, we'll define a schema for that first.

We must also set a description for each field, can provide further hints to ChatGPT for how to guide generation. There is a _lot_ of flexibility here!


In [5]:
class player_character(BaseModel):
    name: str = Field(description="Character name")
    race: str = Field(description="Character race")
    job: str = Field(description="Character class/job")
    story: str = Field(description="Three-sentence character history")
    feats: List[str] = Field(description="Character feats")

An important note: with this new ChatGPT model, the fields are generated _in order_ at runtime according to the schema. Therefore, the order of the fields specified is important! Try to chain information!

Now we can build the schema for the TTRPG we will send to ChatGPT. In this case, we will order the fields such that we generate `description` and then `name`, as the former will be more imaginative and the latter can be infered from it. We will also add a list of player characters using the player character schema above.

Lastly, we will also include a docstring for the schema class; the specifics don't matter but it can provide another editorial hint.


In [6]:
class write_ttrpg_setting(BaseModel):
    """Write a fun and innovative TTRPG"""

    description: str = Field(
        description="Detailed description of the setting in the voice of the DM"
    )
    name: str = Field(description="Name of the setting")
    pcs: List[player_character] = Field(description="Player characters of the TTRPG")

In [7]:
response_structured = ai(
    "Python software development and beach volleyball", output_schema=write_ttrpg_setting
)

# orjson.dumps preserves field order from the ChatGPT API
print(orjson.dumps(response_structured, option=orjson.OPT_INDENT_2).decode())

{
  "description": "Welcome to the sun-kissed shores of Pythos, a land where software development and beach volleyball intertwine. In this unique setting, the power of Python programming and the thrill of competitive sports combine to create an unforgettable adventure. The land of Pythos is known for its pristine beaches, crystal-clear waters, and thriving tech industry. The locals, known as the Code Warriors, rely on their exceptional coding skills to develop cutting-edge software and maintain the digital infrastructure of the realm. But it's not all work and no play in Pythos. The Code Warriors also indulge in their passion for beach volleyball, competing in intense matches with rival teams from neighboring lands. As a player in this TTRPG, you will embark on a journey to master the art of Python software development and become a legendary beach volleyball player. Are you ready to dive into the world of Pythos?",
  "name": "Pythos: Code Warriors",
  "pcs": [
    {
      "name": "Auro

Since the output is structured, we can parse it as we want.

For example, if we just want the setting name:


In [8]:
response_structured["name"]

'Pythos: Code Warriors'

Or if we just the names of the player characters:


In [9]:
[x["name"] for x in response_structured["pcs"]]

['Aurora', 'Blaze']

## Structured Output and Structured Input

Now that we have a schema for a TTRPG setting, we can use the same hints we defined to help generation of a TTRPG adventure!

First, we convert the structured `dict` data to a pydantic object with that schema with `parse_obj`:


In [10]:
input_ttrpg = write_ttrpg_setting.parse_obj(response_structured)

Next, we define a schema for a list of events. To keep things simple, we'll just do **dialogue** and **setting** events. (a proper TTRPG would likely have a more detailed combat system!)

There are a few other helpful object types you can use to control output:

- `Literal`, to force a certain range of values.
- `Union` can be used to have the model select from a set of schema. For example we have one schema for `Dialogue` and one schema for `Setting`: if unioned, the model will use only one of them, which allows for token-saving output.

Lastly, if the `Field(description=...)` pattern is too wordy, you can use `fd` which is a shortcut.


In [25]:
from simpleaichat.utils import fd


class Dialogue(BaseModel):
    character_name: str = fd(description="character name")
    dialogue: str = fd("Dialogue from the character")


class Setting(BaseModel):
    description: str = fd(
        "Detailed setting or event description, e.g. The sun was bright."
    )


class Event(BaseModel):
    type: Literal["setting", "conversation"] = fd(
        "Whether the event is a scene setting or a conversation by an NPC"
    )
    data: Union[Dialogue, Setting] = fd("Event data")


class write_ttrpg_story(BaseModel):
    """Write an award-winning TTRPG story"""

    events: List[Event] = fd("All events in a movie script.")

Lastly, we'll need a new system prompt since we have a different goal.


In [26]:
system_prompt_event = """You are a world-renowned game master (GM) of tabletop role-playing games (RPGs).

Write a complete three-act story in 10 events with a shocking twist ending using the data from the input_ttrpg function. Write the player characters as a TTRPG party fighting against a new evil.

In the second (2nd) event, the party must be formed.

Rules you MUST follow:
- Always write in the style of 80's fantasy novels.
- All names you create must be creative and unique. Always subvert expectations."""

For the final call, we will need the parsed `input_ttrpg` object as the new "prompt", plus the `write_ttrpg_setting` schema used to build it as the `input_schema`.

Putting it all together:


In [27]:
ai_2 = AIChat(system=system_prompt_event, model=model, save_messages=False, api_key=api_key)

response_story = ai_2(
    input_ttrpg, input_schema=write_ttrpg_setting, output_schema=write_ttrpg_story
)

print(orjson.dumps(response_story, option=orjson.OPT_INDENT_2).decode())

{
  "events": [
    {
      "type": "setting",
      "data": {
        "description": "The sun rises over the golden shores of Pythos, casting a warm glow on the city of Pythopolis. The bustling metropolis is a hub of technology and beach volleyball, where the Code Warriors, a group of talented programmers and volleyball enthusiasts, reside. In the heart of the city, the Code Warriors' headquarters stands tall, a symbol of their dedication to both software development and sports. Inside, Aurora, a talented Python developer, and Blaze, a wise volleyball coach, prepare for a fateful meeting that will change their lives forever."
      }
    },
    {
      "type": "conversation",
      "data": {
        "character_name": "Aurora",
        "dialogue": "Blaze, I've been following your coaching career for years. Your innovative techniques have revolutionized the game of beach volleyball. I want to combine my programming skills with my love for volleyball, and I believe you can help me become

Now that we have a structured output, we can output it like a story, with custom and consistent formatting!

In [28]:
c = Console(width=60, highlight=False)

for event in response_story["events"]:
    data = event["data"]
    if event["type"] == "setting":
        c.print(data["description"], style="italic")
    if event["type"] == "conversation":
        c.print(f"[b]{data['character_name']}[/b]: {data['dialogue']}")

The sun rises over the golden shores of Pythos, casting a 
warm glow on the city of Pythopolis. The bustling metropolis
is a hub of technology and beach volleyball, where the Code 
Warriors, a group of talented programmers and volleyball 
enthusiasts, reside. In the heart of the city, the Code 
Warriors' headquarters stands tall, a symbol of their 
dedication to both software development and sports. Inside, 
Aurora, a talented Python developer, and Blaze, a wise 
volleyball coach, prepare for a fateful meeting that will 
change their lives forever.

Aurora: Blaze, I've been following your coaching career for 
years. Your innovative techniques have revolutionized the 
game of beach volleyball. I want to combine my programming 
skills with my love for volleyball, and I believe you can 
help me become the best.

Blaze: Ah, Aurora, I've heard of your coding prowess. Your 
Python skills are legendary in the tech industry. If you're 
willing to put in the work, I can help you unlock the true 
potential of beach volleyball. Together, we can create a new
era of sports technology.

Aurora: I'm ready to dive in, Blaze. I want to use Python to
analyze the game, develop new strategies, and push the 
boundaries of what's possible on the court.

Blaze: Then we shall embark on this journey together, 
Aurora. But first, we must gather a team of like-minded 
individuals who share our passion for both programming and 
beach volleyball.

...

Aurora: Welcome, my fellow Code Warriors! Together, we will 
combine the power of Python programming and the art of beach
volleyball to achieve greatness. Introduce yourselves, and 
let us know how you plan to contribute to our mission.

Samurai: I am Samurai, a master of precision and discipline.
My coding skills are unmatched, and my agility on the 
volleyball court is unparalleled. With my strategic mind and
lightning-fast reflexes, I will ensure victory for our team.

Pixel: Greetings, Code Warriors! I am Pixel, the 
pixel-perfect programmer. My attention to detail and 
creative problem-solving abilities make me a valuable asset 
to any team. On the volleyball court, my quick thinking and 
adaptability will outwit our opponents.

Nebula: I am Nebula, a cosmic coder with a passion for the 
stars and the digital realm. My expertise lies in data 
analysis and visualization. I will harness the power of 
Python to uncover hidden patterns in our opponents' 
strategies and guide us to victory.

Blitz: Greetings, Code Warriors! I am Blitz, the 
lightning-fast developer. My coding speed is unmatched, and 
my agility on the volleyball court is electrifying. With my 
lightning-quick reflexes and powerful spikes, I will leave 
our opponents in awe.

## MIT License

Copyright (c) 2023 Max Woolf

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.